In [1]:
import os
import os.path as osp
import glob 
import json 

root = './data-sewfactory'

In [2]:
sf_root = osp.join(root, 'sewfactory')
sim_root = osp.join(root, 'simimages')

检查文件夹数量是否能对上

In [3]:
sf_folders = os.listdir(sf_root)
sim_folders = os.listdir(sim_root)

set(sf_folders)==set(sim_folders)

True

确保data_props.json的存在

In [4]:
import tqdm 

err_ls = [] 
for fd in tqdm.tqdm(sf_folders):
    dp_path = osp.join(sf_root, fd, 'data_props.json')
    if not os.path.exists(dp_path):
        err_ls.append(dp_path)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 13707/13707 [00:33<00:00, 408.27it/s]


In [5]:
len(err_ls)

0

检查sewfactory文件下的objects目录是不是都只有0和-50两个情况
```
├── objects
│   ├── wb_dress_sleeveless_ZYWMJ5QUQ9_wb_dress_sleeveless_ZYWMJ5QUQ9_0.obj
│   └── wb_dress_sleeveless_ZYWMJ5QUQ9_wb_dress_sleeveless_ZYWMJ5QUQ9_-50.obj
```


In [6]:
import tqdm 

err_ls = [] 
for fd in tqdm.tqdm(sf_folders):
    dp_path = osp.join(sf_root, fd, 'objects')
    objs = os.listdir(dp_path)
    angles = [i.split('_')[-1] for i in objs]
    angles = set(angles)
    if angles != set(['0.obj', '-50.obj']):
        print(fd, 'should be checked')

 43%|██████▍        | 5839/13707 [00:08<00:11, 672.54it/s]

tee_sleeveless_C2EFEDVIB0_wb_pants_straight_U3CBKG8WW1 should be checked


100%|██████████████| 13707/13707 [00:45<00:00, 302.61it/s]


In [8]:
print(os.listdir(osp.join(sf_root, 'tee_sleeveless_C2EFEDVIB0_wb_pants_straight_U3CBKG8WW1','objects')))

['tee_sleeveless_C2EFEDVIB0_tee_sleeveless_C2EFEDVIB0_0.obj', 'wb_pants_straight_U3CBKG8WW1_wb_pants_straight_U3CBKG8WW1_0.obj']


<font color='red'> 有点诡异，但是不知道是不是允许的，暂且搁置，处理掉其他的，但是依然有报错，那再回头处理这个。感觉是很可能是合理的。</font>

-----

objects和poses文件夹联动

In [12]:
err_ls = [] 
for fd in tqdm.tqdm(sf_folders):
    dp_path = osp.join(sf_root, fd, 'objects')
    objs = os.listdir(dp_path)
    angles = [i.split('_')[-1] for i in objs]
    angles = [i.split('.')[0] for i in angles]
    angles = set(angles)
    for angle in angles:
        angle_digits = angle[1:] if angle[0] == '-' else angle  
        angle_zfilled = angle_digits.zfill(4)
        angle_zfilled_with_sign = '-' + angle_zfilled if angle[0] == '-' else angle_zfilled
        body_info_file = osp.join(sf_root, fd, 'poses', angle_zfilled_with_sign+'__body_info.json')
        if not os.path.exists(body_info_file):
            err_ls.append(fd)
            print(fd, 'err with', angle, 'not exist')

 75%|████████▉   | 10269/13707 [00:00<00:00, 51557.60it/s]

tee_sleeveless_CBJCFDXPTH_skirt_2_panels_CR69QH0QJK err with -50 not exist
jumpsuit_sleeveless_LHJ9QN4DLI err with -50 not exist


100%|████████████| 13707/13707 [00:00<00:00, 45232.79it/s]

jumpsuit_sleeveless_T134NEO518 err with -50 not exist


In [13]:
! pwd 

/home/halle/workspace/programs/sewformer


In [14]:
! ls  data-sewfactory/sewfactory/tee_sleeveless_CBJCFDXPTH_skirt_2_panels_CR69QH0QJK/objects && ls data-sewfactory/sewfactory/tee_sleeveless_CBJCFDXPTH_skirt_2_panels_CR69QH0QJK/poses

skirt_2_panels_CR69QH0QJK_skirt_2_panels_CR69QH0QJK_0.obj
skirt_2_panels_CR69QH0QJK_skirt_2_panels_CR69QH0QJK_-50.obj
tee_sleeveless_CBJCFDXPTH_tee_sleeveless_CBJCFDXPTH_0.obj
tee_sleeveless_CBJCFDXPTH_tee_sleeveless_CBJCFDXPTH_-50.obj
0000__body_info.json


跑了一遍训练，恰巧符合报错：

```
[rank0]:     return [self.dataset[self.indices[idx]] for idx in indices]
[rank0]:   File "/workspace/programs/sewformer/Sewformer/data/dataset.py", line 232, in __getitem__
[rank0]:     smpl_joint_pose = json.load(open(smpl_pos_fn, "r"))["pose"]
[rank0]: FileNotFoundError: [Errno 2] No such file or directory: '/workspace/programs/sewformer/data-sewfactory/sewfactory/tee_sleeveless_CBJCFDXPTH_skirt_2_panels_CR69QH0QJK/poses/-0050__body_info.json'
```

从这个报错，找到相关的代码

```
# __init__
                if os.path.exists(os.path.join(self.root_path, folder, "renders")):
                    gt_folder = os.path.join(self.root_path, folder, "static")
                    img_names = [os.path.join(self.sim_root, folder, fn) for fn in os.listdir(os.path.join(self.root_path, folder, "renders")) if fn.endswith(".png")]
                    for img_name in img_names:
                        if os.path.exists(img_name):
                            merge_names = [(img_name, None, gt_folder)]
                            self.datapoints_names += merge_names
                else:
                    gt_folder = os.path.join(self.root_path, folder)
                    merge_names = [(None, None, gt_folder)]
                    self.datapoints_names += merge_names

# __getitem__
        if "use_smpl_loss" in self.config and  self.config["use_smpl_loss"]: # training: True
            smpl_pos_fn = self.get_smpl_pose_fn(datapoint_name, gt_folder)
            smpl_joint_pose = json.load(open(smpl_pos_fn, "r"))["pose"]

# get_smpl_pose_fn
    def get_smpl_pose_fn(self, datapoint_name, gt_folder):
        return os.path.join(os.path.dirname(gt_folder), "poses", os.path.basename(datapoint_name).split("_")[0] + "__body_info.json")
```

核对一个样例的文件结构是否如上预期

![garment_sample](./md_files/files_of_garment_sample.jpg)

In [24]:
bar = tqdm.tqdm(total=len(sim_folders))
need_be_removed = [] 
err_folders_cnt = 0
for i, sfolder in enumerate(sim_folders):
    sfolder_full = osp.join(sim_root, sfolder)
    simages = os.listdir(sfolder_full)
    angles = [i.split('_')[0] for i in simages]
    angles = set(angles)

    poses_folder = osp.join(sf_root, sfolder, 'poses')
    poses_files = os.listdir(poses_folder)
    angles_pose = [i.split('_')[0] for i in poses_files]
    angles_pose = set(angles_pose)
    err_flag = False
    for angle in angles:
        if angle not in angles_pose:
            err_flag = True
            remove_ls = [osp.join(sim_root, sfolder, i) for i in simages if i.split('_')[0] == angle]
            need_be_removed += remove_ls 
    if err_flag:
        err_folders_cnt += 1
    bar.update()
bar.close()
print(err_folders_cnt)
print(len(need_be_removed))

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 13707/13707 [00:00<00:00, 30507.86it/s]

3
72


In [26]:
f = open('sim_remove_list.txt','w', encoding='utf-8')
f.write('\n'.join(need_be_removed))
f.close()

新的报错

```
[rank0]:     spec_dict = self._load_spec_dict(gt_folder)
[rank0]:   File "/workspace/programs/sewformer/Sewformer/data/dataset.py", line 411, in _load_spec_dict
[rank0]:     static_pose = json.load(open(gt_folder + "/static__body_info.json", "r"))
[rank0]: FileNotFoundError: [Errno 2] No such file or directory: '/workspace/programs/sewformer/data-sewfactory/sewfactory/tee_sleeveless_AWFH638TQU_skirt_8_panels_T9V1QXBFD6/static/static__body_info.json'

(base) root@B450M:/workspace/programs/sewformer/data-sewfactory/sewfactory/jumpsuit_sleeveless_T134NEO518/poses# ls
0000__body_info.json

```

有些文件确实是有`static__body_info.json`...

```
(base) halle@B450M:~/workspace/programs/sewformer/data-sewfactory/sewfactory/wb_dress_sleeveless_ZRFG3R9N6C/static$ ls
spec_config.json           static_210_0.png           static_30_0.png            static_back.png
static_0_30cam_pos.json    static_210_30cam_pos.json  static_30_30cam_pos.json   static__body_info.json
static_0_30.png            static_210_30.png          static_30_30.png           static_frontcam_pos.json
```

结合下面这两段代码
```
    def _load_spec_dict(self, gt_folder):
        if gt_folder in self.gt_jsons["spec_dict"]:
            return self.gt_jsons["spec_dict"][gt_folder]
        else:
            # add smpl root at static pose
            static_pose = json.load(open(gt_folder + "/static__body_info.json", "r"))
            static_root = static_pose["trans"]
            # logger.info(f'gt_folder: {gt_folder}')
            try:
                spec_dict = json.load(open(gt_folder + "/spec_config.json", "r"))
            except UnicodeDecodeError as e:
                logger.error(f'{e}, gt_folder: {gt_folder}')
                exit(-1)
            except Exception as e:
                logger.error(f'{e}, gt_folder: {gt_folder}')
                exit(-1)

            for key, val in spec_dict.items():
                logger.debug(f'spec name getting: {val["spec"]}')
                spec = PureWindowsPath(val["spec"]).parts[-1]
                spec_dict[key]["spec"] = spec
                spec_dict[key]["delta"] = np.array(val["delta"]) - np.array(static_root)
            self.gt_jsons["spec_dict"][gt_folder] = spec_dict
            return spec_dict

# _read_pattern

        for key, spec in spec_list.items():
            if gt_folder + "/" + spec in self.gt_jsons["specs"]:
                pattern = self.gt_jsons["specs"][gt_folder + "/" + spec]
            else:
                pattern = NNSewingPattern(
                    gt_folder + "/" + spec, 
                    panel_classifier=self.panel_classifier, 
                    template_name=self.template_name(spec_dict[key]['spec']))
                self.gt_jsons["specs"][gt_folder + "/" + spec] = pattern
            logger.debug(f'spec_path when use: {gt_folder+"/"+spec}')
            patterns.append(pattern)
```

后一段应该是更深一层级的，应该不关事，这里放出，警惕不要搞混。重点放在第一段。

每个`gt_folder`应该会出现多次，第一次应该是走else分支，之后直接走if分支。所以如果<font color='red'>缺失了`static__body_info.json`，那这个文件夹应该就没用了，没有补救方法</font>

In [28]:
err_ls = []
for folder in tqdm.tqdm(sf_folders):
    filepath = osp.join(sf_root, folder, 'static/static__body_info.json')
    if not os.path.exists(filepath):
        err_ls.append(filepath)
print(len(err_ls))

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 13707/13707 [01:08<00:00, 199.76it/s]

2


In [29]:
err_ls

['./data-sewfactory/sewfactory/tee_sleeveless_C2EFEDVIB0_wb_pants_straight_U3CBKG8WW1/static/static__body_info.json',
 './data-sewfactory/sewfactory/tee_sleeveless_AWFH638TQU_skirt_8_panels_T9V1QXBFD6/static/static__body_info.json']